<a href="https://colab.research.google.com/github/Thandeka20/High-spatial-resolution-imagery-vs-high-spectral-imagery/blob/main/Code/SPOT_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import modules
!pip install rasterio
import rasterio
import ee
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
import geemap
import re
import os
from IPython.display import Image, display
from google.colab import drive

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 31.1 MB/s eta 0:00:00


In [4]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='thandeka-skosana')

In [5]:


def save_df_to_drive(df, file_path_in_drive):
    """
    Function that saves a Pandas DataFrame as a csv file to a Google Drive folder specified by a Google Drive file path

    parameters:
      df (pd.DataFrame): The Pandas DataFrame needing to be saved
      file_path_in_drive (str): Google Drive folder file path that will store the CSV file
    """
    try:
        # Ensure the destination directory exists
        drive.mount('/content/drive')  # Mount Google Drive to access files

        # Get the absolute path in Google Drive
        absolute_path_in_drive = os.path.join('/content/drive/My Drive', file_path_in_drive)

        # Ensure the destination directory exists in Google Drive
        os.makedirs(os.path.dirname(absolute_path_in_drive), exist_ok=True)

        # Save the DataFrame to the specified file path in Google Drive
        df.to_csv(absolute_path_in_drive, index=False)

        print(f"DataFrame saved to Google Drive at '{absolute_path_in_drive}'")

    except Exception as e:
        print("An error occurred:", str(e))

# Example usage:
# Assuming your file_path_in_drive is 'YourFolder/YourFile.csv'
# Make sure to change it according to your actual folder structure
#save_df_to_drive(your_dataframe, 'YourFolder/YourFile.csv')


Get SPOT-7 Image from GEE

In [ ]:
# Define the image
#image = ee.Image('projects/mapwaps-sabiecroc/assets/SPOT7')

In [6]:
#visualize the SPOT-7 scene
# Define the bands to use for visualization
bands = ee.List(["B4", "B3", "B2"])

# Define visualization parameters
vis_params = {
    'bands': ['b1', 'b2', 'b3'],
    'min': 0,
    'max': 3200,
}

# Example: Display an image using the defined visualization parameters
image = ee.Image('projects/mapwaps-sabiecroc/assets/SPOT7')
display_image = Image(url=image.visualize(**vis_params).getThumbURL({'dimensions': '500x500'}))
display(display_image)

Extracting SPOT 7 bands

In [9]:

def get_SPOT_7_bands(image_id, latitude, longitude):
    """
    Function that extracts the spectral bands from a SPOT 7 satellite image of a specified location.

    Parameters:
        image_id (str): Unique SPOT 7 image ID.
        latitude (float): Latitude coordinate of the desired location.
        longitude (float): Longitude coordinate of the desired location.

    Returns:
        band_dict (dict): Dictionary of multispectral band values for a specified location with labels b1-b4.
    """
    try:
        # Load the SPOT 7 image by its ID
        spot7_image = ee.Image(image_id)

        # Define a point geometry for the specified latitude and longitude
        point_geometry = ee.Geometry.Point([longitude, latitude])

        # Use the .sample() method to extract pixel values at the specified geometry
        # This will create a feature collection containing the pixel values
        pixel_values = spot7_image.sample(point_geometry, 6)  # Adjust scale as needed

        # Extract band values from the feature collection and add them to the dictionary
        band_dict = {}
        band_names = spot7_image.bandNames().getInfo()

        for band_name in band_names:
            band_value = pixel_values.first().get(band_name)
            band_dict[band_name] = ee.Number(band_value).getInfo()

        return band_dict

    except ee.EEException as e:
        return {"error": "An Earth Engine exception occurred: " + str(e)}
    except Exception as e:
        return {"error": "An unexpected error occurred: " + str(e)}

# -----------------------------------------------------------------------------------------------------------------
# Example usage:
spot7_image_id = 'projects/mapwaps-sabiecroc/assets/SPOT7'
latitude = -25.12
longitude = 30.88

band_data = get_SPOT_7_bands(spot7_image_id, latitude, longitude)
print(band_data)


{'error': "An Earth Engine exception occurred: Element.get: Parameter 'object' is required."}
